# Understanding ELF

## Example: Creating Really Teensy ELF Executables

Reference: 

- http://www.muppetlabs.com/~breadbox/software/tiny/teensy.html
- https://cjting.me/2020/12/10/tiny-x64-helloworld/#step2-optimization 

In [1]:
%%writefile tiny.c
#include <stdio.h>

int main() {
  printf("hello, world\n");
  return 0;
}

Writing tiny.c


In [2]:
!gcc -Wall tiny.c
!ls -l a.out
# Strip the executable
!gcc -Wall -s tiny.c
!ls -l a.out

-rwxr-xr-x 1 root root 16696 Jan  3 20:54 a.out
-rwxr-xr-x 1 root root 14472 Jan  3 20:54 a.out


In [3]:
%%writefile tiny.c
#include <stdio.h>
#include <unistd.h>

int nomain()
{
  printf("hello, world\n");
  _exit(0);
}

Overwriting tiny.c


> Use `_exit` to abort the child program when the `exec` fails, because in this situation, the child process may interfere with the parent process' external data (files) by calling its `atexit` handlers, calling its signal handlers, and/or flushing buffers.
>
> See [here](https://stackoverflow.com/questions/5422831/what-is-the-difference-between-using-exit-exit-in-a-conventional-linux-fo) for details.

In [4]:
!gcc -e nomain -nostartfiles tiny.c
!ls -l a.out

-rwxr-xr-x 1 root root 14800 Jan  3 20:54 a.out


In [5]:
%%writefile tiny.c
char *str = "hello, world\n";

void myprint()
{
  asm("movq $1, %%rax \n"
      "movq $1, %%rdi \n"
      "movq %0, %%rsi \n"
      "movq $13, %%rdx \n"
      "syscall \n"
      : // no output
      : "r"(str)
      : "rax", "rdi", "rsi", "rdx");
}

void myexit()
{
  asm("movq $60, %rax \n"
      "xor %rdi, %rdi \n"
      "syscall \n");
}

int nomain()
{
  myprint();
  myexit();
}

Overwriting tiny.c


In [6]:
!gcc -e nomain -nostartfiles tiny.c
!ls -l a.out
!readelf -S -W a.out

-rwxr-xr-x 1 root root 14384 Jan  3 20:54 a.out
There are 18 section headers, starting at offset 0x33b0:

Section Headers:
  [Nr] Name              Type            Address          Off    Size   ES Flg Lk Inf Al
  [ 0]                   NULL            0000000000000000 000000 000000 00      0   0  0
  [ 1] .interp           PROGBITS        0000000000000318 000318 00001c 00   A  0   0  1
  [ 2] .note.gnu.property NOTE            0000000000000338 000338 000020 00   A  0   0  8
  [ 3] .note.gnu.build-id NOTE            0000000000000358 000358 000024 00   A  0   0  4
  [ 4] .gnu.hash         GNU_HASH        0000000000000380 000380 00001c 00   A  5   0  8
  [ 5] .dynsym           DYNSYM          00000000000003a0 0003a0 000018 18   A  6   1  8
  [ 6] .dynstr           STRTAB          00000000000003b8 0003b8 000001 00   A  0   0  1
  [ 7] .rela.dyn         RELA            00000000000003c0 0003c0 000018 18   A  5   0  8
  [ 8] .text             PROGBITS        0000000000001000 001000 000062 00

In [7]:
%%writefile link.lds
ENTRY(nomain)

SECTIONS
{
  . = 0x8048000 + SIZEOF_HEADERS;
  tiny : { *(.text) *(.data*) *(.rodata*) }
  /DISCARD/ : { *(*) }
}

Writing link.lds


In [8]:
!gcc -T link.lds -e nomain -nostartfiles tiny.c
!ls -l a.out

/usr/bin/ld: warning: .note.gnu.build-id section discarded, --build-id ignored
-rwxr-xr-x 1 root root 920 Jan  3 20:54 a.out


In [9]:
%%writefile tiny.asm
section .data
message: db "hello, world", 0xa

section .text

global nomain
nomain:
  mov rax, 1
  mov rdi, 1
  mov rsi, message
  mov rdx, 13
  syscall
  mov rax, 60
  xor rdi, rdi
  syscall

Writing tiny.asm


In [10]:
!nasm -f elf64 tiny.asm
!gcc -T link.lds -e nomain -nostartfiles tiny.o
!ls -l a.out

/usr/bin/ld: warning: .note.gnu.build-id section discarded, --build-id ignored
-rwxr-xr-x 1 root root 736 Jan  3 20:54 a.out


In [11]:
%%writefile tiny.asm
BITS 64
  org 0x400000

ehdr:           ; Elf64_Ehdr
  db 0x7f, "ELF", 2, 1, 1, 0 ; e_ident
  times 8 db 0
  dw  2         ; e_type
  dw  0x3e      ; e_machine
  dd  1         ; e_version
  dq  _start    ; e_entry
; $$ evaluates to the beginning of the current section
; so you can tell how far into the section you are by using ($-$$).
  dq  phdr - $$ ; e_phoff
  dq  0         ; e_shoff
  dd  0         ; e_flags
  dw  ehdrsize  ; e_ehsize
  dw  phdrsize  ; e_phentsize
  dw  1         ; e_phnum
  dw  0         ; e_shentsize
  dw  0         ; e_shnum
  dw  0         ; e_shstrndx
; EQU defines a symbol to a given constant value
ehdrsize  equ  $ - ehdr

phdr:           ; Elf64_Phdr
  dd  1         ; p_type
  dd  5         ; p_flags
  dq  0         ; p_offset
  dq  $$        ; p_vaddr
  dq  $$        ; p_paddr
  dq  filesize  ; p_filesz
  dq  filesize  ; p_memsz
  dq  0x1000    ; p_align
phdrsize  equ  $ - phdr

_start:
  mov rax, 1
  mov rdi, 1
  mov rsi, message
  mov rdx, 13
  syscall
  mov rax, 60
  xor rdi, rdi
  syscall

message: db "hello, world", 0xa

filesize  equ  $ - $$

Overwriting tiny.asm


In [12]:
!nasm -f bin tiny.asm
!ls -l tiny
!chmod +x tiny
!./tiny

-rw-r--r-- 1 root root 170 Jan  3 20:54 tiny
hello, world


In [13]:
!rm tiny.c tiny.o tiny.asm a.out tiny link.lds